In [68]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

import cv2
import skimage
from skimage.feature import hog

from sklearn.model_selection import train_test_split

import os
import gzip
import pickle
from tqdm.auto import tqdm
from ast import literal_eval
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report



pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [69]:
images = np.load("image.npy")
image_labels = np.load("image_category.npy")

In [70]:
np.unique(image_labels)

array([ 1, 17, 18, 27, 64])

In [73]:
def extract_HOG_features(data):
    num_samples = data.shape[0]
    hog_features = []
    for i in tqdm(range(num_samples)):
        img = data[i]
        feature = hog(img)
        hog_features.append(feature)
    return np.array(hog_features)

def model_train(
    images,
    image_labels,
    hidden_layer_sizes=(256, 128, 5),
    verbose = False
):
    
    train_features, test_features, train_labels, test_labels = train_test_split(images, image_labels)
    
    train_features = extract_HOG_features(train_features)
    test_features = extract_HOG_features(test_features)
    
    clf = MLPClassifier(solver='lbfgs', 
                    activation='relu', 
                    alpha=50, 
                    hidden_layer_sizes=hidden_layer_sizes, 
                    random_state=1, 
                    max_iter=500,
                    early_stopping=True,
                    verbose=verbose)
    
    clf.fit(train_features, train_labels)
    
    pred = clf.predict(train_features)
    train_accuracy = np.mean(pred == train_labels)

    pred = clf.predict(test_features)
    proba = clf.predict_proba(test_features)
    test_accuracy = np.mean(pred == test_labels)
    
    print("Training accuracy: {}".format(train_accuracy))
    print("Testing accuracy: {}".format(test_accuracy))
    print(classification_report(test_labels, pred))
    
    return clf

In [72]:
clf = model_train(images, image_labels)

  0%|          | 0/2224 [00:00<?, ?it/s]

  0%|          | 0/742 [00:00<?, ?it/s]

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =      4098083     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.45688D+00    |proj g|=  1.57983D-01


 This problem is unconstrained.



At iterate    1    f=  9.02339D+00    |proj g|=  5.04775D-01

At iterate    2    f=  8.58290D+00    |proj g|=  2.59897D-01

At iterate    3    f=  7.61945D+00    |proj g|=  5.71364D-01

At iterate    4    f=  6.78844D+00    |proj g|=  5.09340D-01

At iterate    5    f=  4.46828D+00    |proj g|=  2.13986D-01

At iterate    6    f=  4.06366D+00    |proj g|=  5.81856D-01

At iterate    7    f=  3.44890D+00    |proj g|=  1.74270D+00

At iterate    8    f=  3.43888D+00    |proj g|=  3.72437D+00

At iterate    9    f=  3.13389D+00    |proj g|=  2.63861D+00

At iterate   10    f=  2.83178D+00    |proj g|=  3.32864D+00

At iterate   11    f=  2.56499D+00    |proj g|=  7.43106D-01

At iterate   12    f=  2.29234D+00    |proj g|=  1.09301D+00

At iterate   13    f=  2.11500D+00    |proj g|=  7.59147D-01

At iterate   14    f=  2.05017D+00    |proj g|=  5.32531D-01

At iterate   15    f=  2.00074D+00    |proj g|=  3.59090D-01

At iterate   16    f=  1.97355D+00    |proj g|=  3.23547D-01

At iter

/Users/grantwilson/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Training accuracy: 0.9784172661870504
Testing accuracy: 0.34097035040431267
              precision    recall  f1-score   support

           1       0.28      0.33      0.31       177
          17       0.18      0.16      0.17        96
          18       0.20      0.19      0.20       106
          27       0.49      0.44      0.46       190
          64       0.42      0.44      0.43       173

    accuracy                           0.34       742
   macro avg       0.31      0.31      0.31       742
weighted avg       0.34      0.34      0.34       742

